In [ ]:
## 모델
from langchain.chat_models import ChatOpenAI
## 포멧팅 템플릿팅
from langchain.prompts import PromptTemplate 
## 모델에게 어떻게 대답해야하는지 템플릿을 만들어주는 것 
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate 
## 응답 스트리밍 콜백
from langchain.callbacks import StreamingStdOutCallbackHandler
## 채팅 프롬프트 -- 역할을 설정해서 대화형으로 생성
from langchain.prompts import ChatPromptTemplate
## 설정 값에 따라 prompt에 맞는 예제를 골라줌
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector 


## gpt-3.5-turbo
chat = ChatOpenAI(
    temperature=0.1,  ## 0 ~ 1 1에 가까울수록 창의적
    streaming=True, ## 중간 내용 로깅 위한 스트리밍 설정 True
    callbacks=[
        StreamingStdOutCallbackHandler() ## 중간 내용 로깅
    ]
)

t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country="France")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template = """
    Human : {question},
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}",
    input_variables=["country"]
)

prompt.format(country="germany")

chain = prompt | chat

chain.invoke({
    "country" : "korea"
})

In [ ]:
chat_examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_chat_prompt = ChatPromptTemplate.from_messages([
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}")
])

example_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_chat_prompt,
    examples=chat_examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are geography expert. you give short answers"),
        example_chat_prompt,
        ("human", "what do you know about {country}?")
    ]
)

chain = final_prompt | chat 

chain.invoke({"country" : "south korea"})

In [ ]:
example_selector=LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=20
)

prompt1 = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix ="Human: what do you know about {country}",
    input_variables=["country"]
)

prompt1.format(country="brazil")

In [ ]:
## 예제 중에 랜덤으로 뽑아서 예제로 학습시키기
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_selector = RandomExampleSelector(
    examples=examples
)

prompt2 = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix ="Human: what do you know about {country}",
    input_variables=["country"]
)

prompt2.format(country="brazil")

In [ ]:
## 디스크에서 prmopt templates 가져오기
from langchain.prompts import load_prompt

p = load_prompt("./prompt.json")
y = load_prompt("./prompt.yaml")

y.format(country="korea")

In [ ]:
## 여러개의 프롬프트 합치기
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts =[
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts
)



chain = full_prompt | chat 

chain.invoke({
    "character" : "pirate",
    "example_question":"what is your location",
    "example_answer":"arrrg? that is a secret!! argg argg",
    "question" : "what is you favoriate food?"
})


In [ ]:
## 4.5 Chaching
## 같은 질문 받으면 캐싱하는 것 
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1
)

chat.predict("How do you make italian pasta")

In [ ]:
chat.predict("How do you make italian pasta")

In [ ]:
## 4.6 Serialization 
## usage view
from langchain.callbacks import get_openai_callback

with get_openai_callback() as usage:
    a = chat.predict("what is the recipie for soju")
    b = chat.predict("what is recepie for bread")
    print(a,b,"\n")
    ## 사용량 출력
    print(usage)

In [40]:
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k"
)
chat.save("model.json")

chat1 = load_llm("model.json")
print(chat1)

OpenAIChat
Params: {'model_name': 'gpt-3.5-turbo-16k', 'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


/Users/jocelynseong/NodeProjects/FULLSTACK-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jocelynseong/NodeProjects/FULLSTACK-GPT/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
